# Instruction

In this assignment, you will analyze the political tweets during the campaign of the 2017 UK General Election. The Twitter database, in SQLite format, is available at the link below (it's zipped, so you need to decompress before working on it).

https://www.dropbox.com/s/jh40sd8xebjg9pn/ge_2017.sqlite.zip?dl=1


## Data description

When I collected the data, I saved the streamed Twitter data every 15 minutes. The condition of streaming is to get all tweets from the candidates and party official accounts and replies/retweets to the tweets from these accounts. I randomly selected 10 percent of the data files for this assignment.

After selecting the data I created two tables from the data, one for twitter content, and another for user information. All the field names are original names from Twitter JSON, so it should be easy to understand the contents. I added one field to the user table, which is `screen_name_in` where the value 1 indicates the election candidates' or party official accounts. 0 indicates otherwise.

# Load packages

In [27]:
import pandas as pd
import sqlite3

In [28]:
!wget https://www.dropbox.com/s/jh40sd8xebjg9pn/ge_2017.sqlite.zip?dl=1

--2023-12-31 10:55:41--  https://www.dropbox.com/s/jh40sd8xebjg9pn/ge_2017.sqlite.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/jh40sd8xebjg9pn/ge_2017.sqlite.zip [following]
--2023-12-31 10:55:42--  https://www.dropbox.com/s/dl/jh40sd8xebjg9pn/ge_2017.sqlite.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce75ee9dad0860fd17b016e5d95.dl.dropboxusercontent.com/cd/0/get/CKe2LmAXyyYpnxrIdPuDSO3zMu66XGc3aIGJ4Tlo4jA8HzcLRoodVoDL8HvgKvM_TCnIpAmLCIB7ztMw9Z68mGUIgUO4x9vY5Vx_THx5xmWcqkc61ppUNf73VjHdvPGpniWHQ3n_X5ZuqOUha63vzRzz/file?dl=1# [following]
--2023-12-31 10:55:42--  https://uce75ee9dad0860fd17b016e5d95.dl.dropboxusercontent.com/cd/0/get/CKe2LmAXyyYpnxrIdPuDSO3zMu66XGc3aIGJ4Tlo4jA8HzcLRoodVoDL8HvgKvM_TCnIpAmLCIB7ztMw9Z

In [29]:
!unzip ge_2017.sqlite.zip\?dl\=1

Archive:  ge_2017.sqlite.zip?dl=1
replace ge_2017.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Connect to the database (2 percent)

In [30]:
con_ge = sqlite3.connect('ge_2017.sqlite')

# Check the database (5 percent)

How many tweets are in the tweet table? How many users in the users table?

In [31]:
df_tab_info = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", con_ge)
df_tab_info
# using pd.read to read the sqlite master table. This tells me how many tables are in this dataset. There are 2

,type,name,tbl_name,rootpage,sql
0,table,tweets,tweets,2,"CREATE TABLE `tweets` (\n `doc_id` TEXT,\n `..."
1,table,users,users,21681,"CREATE TABLE `users` (\n `verified` INTEGER,\..."


In [32]:
print(df_tab_info['sql'][0])
#checking fields in tweets table for future referecne

CREATE TABLE `tweets` (
  `doc_id` TEXT,
  `text` TEXT,
  `retweet_count` REAL,
  `favorite_count` REAL,
  `favorited` INTEGER,
  `truncated` INTEGER,
  `id_str` TEXT,
  `in_reply_to_screen_name` TEXT,
  `source` TEXT,
  `retweeted` INTEGER,
  `created_at` TEXT,
  `in_reply_to_status_id_str` TEXT,
  `in_reply_to_user_id_str` TEXT,
  `lang` TEXT,
  `listed_count` REAL,
  `user_id_str` TEXT,
  `country_code` TEXT,
  `country` TEXT,
  `place_type` TEXT,
  `full_name` TEXT,
  `place_name` TEXT,
  `place_id` TEXT,
  `place_lat` REAL,
  `place_lon` REAL,
  `lat` REAL,
  `lon` REAL,
  `expanded_url` TEXT,
  `url` TEXT,
  `created_at_dt` REAL
)


In [33]:
pd.read_sql_query("SELECT * FROM tweets LIMIT 5", con_ge)

,doc_id,text,retweet_count,favorite_count,favorited,truncated,id_str,in_reply_to_screen_name,source,retweeted,...,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url,created_at_dt
0,tw_json_6_201705261745.json.1,RT @BenRich66: If you work in tech and are bac...,0.0,0.0,0,0,868157285226491905,None,"<a href=""http://twitter.com/download/iphone"" r...",0,...,None,None,None,None,None,None,None,http://www.libdems.org.uk/tech-backing-libdems,https://t.co/tWDj1dWU6R,1.495820e+09
1,tw_json_6_201705261745.json.2,RT @AngelaRayner: Whilst polls can fluctuate i...,0.0,0.0,0,0,868157291270467584,None,"<a href=""https://www.chicmi.com/"" rel=""nofollo...",0,...,None,None,None,None,None,None,None,None,None,1.495820e+09
2,tw_json_6_201705261745.json.3,RT @aw_bell: We must defend our freedoms from ...,0.0,0.0,0,0,868157295083102210,None,"<a href=""http://twitter.com/download/android"" ...",0,...,None,None,None,None,None,None,None,None,None,1.495820e+09
3,tw_json_6_201705261745.json.4,RT @UKLabour: Education should be available to...,0.0,0.0,0,0,868157295938633729,None,"<a href=""http://twitter.com/download/iphone"" r...",0,...,None,None,None,None,None,None,None,None,None,1.495820e+09
4,tw_json_6_201705261745.json.5,RT @RichardBurgon: The stakes are high in this...,0.0,0.0,0,0,868157297750671361,None,"<a href=""http://twitter.com/#!/download/ipad"" ...",0,...,None,None,None,None,None,None,None,https://twitter.com/i/web/status/8681572979813...,,1.495820e+09


In [34]:
pd.read_sql_query("SELECT * FROM users LIMIT 5", con_ge)

,verified,location,user_id_str,description,geo_enabled,user_created_at,statuses_count,followers_count,favourites_count,protected,user_url,name,time_zone,user_lang,utc_offset,friends_count,screen_name,user_created_at_dt,screen_name_in
0,0,Mostly in Liverpool,20255744,"Leader Liverpool Lib Dems, Lib Dem Spokesperso...",1,Fri Feb 06 17:54:34 +0000 2009,42343.0,3914.0,158.0,0,http://richardkemp.org.uk,Richard Kemp CBE,Amsterdam,en,7200.0,632.0,cllrkemp,1.233943e+09,1
1,0,"London, UK",1871497250,I tweet the top stories that Westminster are t...,0,Mon Sep 16 12:33:23 +0000 2013,9262.0,258.0,23407.0,0,None,govmixx,None,en,NaN,865.0,govmixx,1.379335e+09,0
2,0,"Easton/Lawrence Hill, Bristol",427599291,Field Organiser for @TheGreenParty,1,Sat Dec 03 18:43:42 +0000 2011,39943.0,5595.0,2390.0,0,http://robtelford.com,Rob Telford,London,en,3600.0,4819.0,GreenRobTelford,1.322938e+09,0
3,0,"Bury, England",317402880,"Chippy Yorkshireman, Labour & Remain supporter...",0,Tue Jun 14 21:51:59 +0000 2011,4060.0,145.0,2937.0,0,None,Manumission,None,en,NaN,312.0,lockforward4,1.308088e+09,0
4,0,None,4309416083,None,0,Sat Nov 28 17:43:36 +0000 2015,26420.0,270.0,14127.0,0,None,Donald Duck,London,en-gb,3600.0,435.0,maninthestreet9,1.448733e+09,0


In [35]:
print(df_tab_info['sql'][1])
#checking fields in users table for future reference

CREATE TABLE `users` (
  `verified` INTEGER,
  `location` TEXT,
  `user_id_str` TEXT,
  `description` TEXT,
  `geo_enabled` INTEGER,
  `user_created_at` TEXT,
  `statuses_count` REAL,
  `followers_count` REAL,
  `favourites_count` REAL,
  `protected` INTEGER,
  `user_url` TEXT,
  `name` TEXT,
  `time_zone` TEXT,
  `user_lang` TEXT,
  `utc_offset` REAL,
  `friends_count` REAL,
  `screen_name` TEXT,
  `user_created_at_dt` REAL,
  `screen_name_in` INTEGER
)


In [36]:
pd.read_sql_query("""
SELECT COUNT (*)
FROM tweets
""", con_ge)
# From the previous query, I know have the names of the tables, thus I can make queries on the individual tabkes

,COUNT (*)
0,221711


There are 221,711 tweets in the table

In [37]:
pd.read_sql_query("""
SELECT COUNT (*)
FROM users
""", con_ge)

,COUNT (*)
0,91256


There are 91,256 users in the table

# Description of the data (21 percent, 3 per question)

Answer the following questions (try to do the most computation in the query, if you do some work in Python, you will only get partial credit):

1. Which screen_name has the highest count of tweets?
2. Who has the highest number of followers?
3. Among politicians, who has the highest number of followers?
4. Which tweet has the earliest timestamp in the data? Which is the latest?
5. Who were the top ten most replied accounts? How many times?
6. How many tweets with the word Brexit?
7. How many tweets have geolocation information (`lat` or `lon` value)?


In [38]:
# 1. Max count of tweets
# Using field status_count as number of tweets
pd.read_sql_query("SELECT MAX(statuses_count) FROM users", con_ge)

,MAX(statuses_count)
0,3256136.0


In [39]:
#finding the exact screen name with that tweet count

pd.read_sql_query("""
SELECT*
FROM users
WHERE statuses_count = 3256136.0
""", con_ge)

,verified,location,user_id_str,description,geo_enabled,user_created_at,statuses_count,followers_count,favourites_count,protected,user_url,name,time_zone,user_lang,utc_offset,friends_count,screen_name,user_created_at_dt,screen_name_in
0,0,愛媛県 Japan,93196638,MacBook Proユーザ。音楽とサッカー（とその他スポーツ）と猫関係のツイート、リツイー...,0,Sat Nov 28 14:49:44 +0000 2009,3256136.0,4768.0,68.0,0,http://hisashitoshima.cocolog-nifty.com,としま,Tokyo,en,32400.0,550.0,t_hisashi,1.259420e+09,0


The screen name with the highest tweet count of 3,256,136 is t_hisashi.

In [41]:
# 2. Highest number of followers

pd.read_sql_query("SELECT MAX(followers_count) FROM users", con_ge)

,MAX(followers_count)
0,12419966.0


In [42]:
# Finding the specific user with the highest no. of followers

pd.read_sql_query("""
SELECT *
FROM users
WHERE followers_count = 12419966.0
""", con_ge)

,verified,location,user_id_str,description,geo_enabled,user_created_at,statuses_count,followers_count,favourites_count,protected,user_url,name,time_zone,user_lang,utc_offset,friends_count,screen_name,user_created_at_dt,screen_name_in
0,1,Earth,20015311,"Godless Ape. Comedian, Actor, Screenwriter, Mo...",1,Wed Feb 04 00:00:12 +0000 2009,43211.0,12419966.0,23623.0,0,http://www.rickygervais.com,Ricky Gervais,London,en,3600.0,1156.0,rickygervais,1.233706e+09,0


Ricky Gervais has the highest number of followers, which is 12,419,966.
---



In [43]:
# 3. Highest followers among the politicians
# here, I must filter the results using the added column screen_name_in the users table.
# I seek the highest number of followers where the screen_name_in value is 1 (denotes politician)

pd.read_sql_query("""
SELECT MAX(followers_count)
FROM users
WHERE screen_name_in = 1
""", con_ge)

,MAX(followers_count)
0,968629.0


In [44]:
# Finding the specific politian with highers no. of followers

pd.read_sql_query("""
SELECT *
FROM users
WHERE followers_count = 968629.0
""", con_ge)

,verified,location,user_id_str,description,geo_enabled,user_created_at,statuses_count,followers_count,favourites_count,protected,user_url,name,time_zone,user_lang,utc_offset,friends_count,screen_name,user_created_at_dt,screen_name_in
0,1,UK,117777690,Leader of the Labour Party. Support our #GE201...,0,Fri Feb 26 15:45:23 +0000 2010,7878.0,968629.0,125.0,0,http://www.labour.org.uk/volunteering,Jeremy Corbyn,London,en,3600.0,2424.0,jeremycorbyn,1.267199e+09,1


Jeremy Corbyn is the politician with the highest number of followers, which is 968,629.

In [45]:
#4. Tweet with earliest timestamp
# using the created_at_dt field in the tweets table
# using MIN function to show the earliest number

pd.read_sql_query("SELECT MIN(created_at_dt) FROM tweets", con_ge)

,MIN(created_at_dt)
0,1.495820e+09


In [46]:
# the specific tweet

pd.read_sql_query("""
SELECT *
FROM tweets
WHERE created_at_dt = 1.495820e+09
""", con_ge)

,doc_id,text,retweet_count,favorite_count,favorited,truncated,id_str,in_reply_to_screen_name,source,retweeted,...,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url,created_at_dt
0,tw_json_6_201705261745.json.304,"RT @daily_politics: ""Maybe we should just stop...",0.0,0.0,0,0,868158123072159744,None,"<a href=""http://twitter.com/download/android"" ...",0,...,None,None,None,None,None,None,None,https://twitter.com/MammothWhale/status/868146...,,1.495820e+09


In [47]:
# 4. Tweet with latest timestamp
# using MAX function to show the latest number

pd.read_sql_query("SELECT MAX(created_at_dt) FROM tweets", con_ge)

,MAX(created_at_dt)
0,1.496910e+09


In [54]:
# the specific tweet

pd.read_sql_query("""
SELECT *
FROM tweets
WHERE created_at_dt = 1.496910e+09
""", con_ge)


,doc_id,text,retweet_count,favorite_count,favorited,truncated,id_str,in_reply_to_screen_name,source,retweeted,...,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url,created_at_dt


In [ ]:
# 5. Top ten most replied accounts



In [55]:
#6. No. of tweets with word Brexit

pd.read_sql_query("""
SELECT COUNT (*)
 FROM tweets
 WHERE text LIKE '%Brexit%'
 """, con_ge)

,COUNT (*)
0,9028


There are 9,028 tweets that mention Brexit

In [52]:
#7. How many tweets have geolocation information (lat or lon value)?

pd.read_sql_query("""
SELECT COUNT (*)
FROM tweets
WHERE lat
""", con_ge)


,COUNT (*)
0,35


In [53]:
pd.read_sql_query("""
SELECT COUNT (*)
FROM tweets
WHERE lon
""", con_ge)

,COUNT (*)
0,35


35 tweets have geolocation information.

# Use of hashtags (10 percent)

In this section, you will work on hashtags in tweets. If you are not sure what is hashtags, please refer to: https://business.twitter.com/en/blog/how-to-create-and-use-hashtags.html

### Tasks:

1. Think about the ways to extract hashtags from the Twitter text. The goal here is to create a two-column DataFrame where the first column is tweet id (`id_str`) and the second column is the content of the hashtag (e.g. `#ge2017`, `#conservative`).

2. Using the dataframe and the database, find the following:
  - What are the 10 most popular hashtags?
  - What are the 10 most popular hashtags by political Twitter accounts (i.e. election candidates/parties)? What about non-politician/parties account?




In [57]:
pd.read_sql_query("SELECT text FROM tweets", con_ge)


,text
0,RT @BenRich66: If you work in tech and are bac...
1,RT @AngelaRayner: Whilst polls can fluctuate i...
2,RT @aw_bell: We must defend our freedoms from ...
3,RT @UKLabour: Education should be available to...
4,RT @RichardBurgon: The stakes are high in this...
...,...
221706,RT @jeremycorbyn: We've left no place and no s...
221707,@jaybn1 Thank you so much Jay x
221708,@jeremycorbyn @UKLabour Thank you Jeremy and t...
221709,RT @Conservatives: TM: We cannot and must not ...


In [39]:
pd.read_sql_query("""
SELECT *
 FROM tweets
 WHERE text LIKE '%#%'
 """, con_ge)

,doc_id,text,retweet_count,favorite_count,favorited,truncated,id_str,in_reply_to_screen_name,source,retweeted,...,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url,created_at_dt
0,tw_json_6_201705261745.json.5,RT @RichardBurgon: The stakes are high in this...,0.0,0.0,0,0,868157297750671361,None,"<a href=""http://twitter.com/#!/download/ipad"" ...",0,...,None,None,None,NaN,NaN,NaN,NaN,https://twitter.com/i/web/status/8681572979813...,,1.495820e+09
1,tw_json_6_201705261745.json.9,Less than 2 weeks until #GE2017. Team #Cherry4...,0.0,0.0,0,0,868157303651840000,None,"<a href=""https://ads.twitter.com"" rel=""nofollo...",0,...,None,None,None,NaN,NaN,NaN,NaN,None,None,1.495820e+09
2,tw_json_6_201705261745.json.10,Why people are voting Labour in Harborough #GE...,0.0,0.0,0,0,868157299021553664,None,"<a href=""https://twittimer.com"" rel=""nofollow""...",0,...,None,None,None,NaN,NaN,NaN,NaN,None,None,1.495820e+09
3,tw_json_6_201705261745.json.11,RT @Conservatives: Your vote at #GE2017 will h...,0.0,0.0,0,0,868157309171752961,None,"<a href=""http://twitter.com/download/android"" ...",0,...,None,None,None,NaN,NaN,NaN,NaN,None,None,1.495820e+09
4,tw_json_6_201705261745.json.16,RT @hazel_thorpe: Self interest ? @OliHenman @...,0.0,0.0,0,0,868157323465953282,None,"<a href=""http://twitter.com/download/android"" ...",0,...,None,None,None,NaN,NaN,NaN,NaN,None,None,1.495820e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65964,tw_json_6_201706080815.json.5642,RT @UKLabour: #VoteLabour today for a fairer B...,0.0,0.0,0,0,872728646183784448,None,"<a href=""http://twitter.com/download/android"" ...",0,...,None,None,None,NaN,NaN,NaN,NaN,https://twitter.com/LadPolitics/status/8723970...,https://t.co/953WYc2p0c,1.496910e+09
65965,tw_json_6_201706080815.json.5643,RT @ChriswMP: The vile disgusting S*n is at it...,0.0,0.0,0,0,872728646854864897,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",0,...,None,None,None,NaN,NaN,NaN,NaN,https://twitter.com/marcjacobleo/status/872727...,https://t.co/0pkGkNSaYE,1.496910e+09
65966,tw_json_6_201706080815.json.5650,@UKLabour Only a vote for labour can stop the ...,0.0,0.0,0,0,872728651716067328,UKLabour,"<a href=""http://twitter.com/#!/download/ipad"" ...",0,...,None,None,None,NaN,NaN,NaN,NaN,None,None,1.496910e+09
65967,tw_json_6_201706080815.json.5651,RT @Stortlibdems: Vote Mark Argent for Hertfor...,0.0,0.0,0,0,872728652127055873,None,"<a href=""http://twitter.com/download/iphone"" r...",0,...,None,None,None,NaN,NaN,NaN,NaN,None,None,1.496910e+09


In [40]:
pd.read_sql_query("""
SELECT DISTINCT
 FROM tweets
 WHERE text LIKE '%#%'
 """, con_ge)

DatabaseError: ignored

# Further data exploration (12 percent)

In this section, you are asked to make further exploration of the data, using the additional data in the repository below (`cand_data.csv`). The data includes the candidate's name, party affiliation, incumbent MP, and screenName. Match this data with the Twitter data, explore the data and describe what you have found using tables, visuals, and/or models.


In [ ]:
!git clone  https://github.com/University-of-Essex-Dept-of-Government/GV918-UK-politics-data


I suggest a few questions you can answer (These are just suggestions; you don't have to answer all of them, or you can answer completely different questions):

- Are there any differences in the use of particular hashtags? Which hashtags are more popular among Conservative/Labour candidates? What is the possible reasons for the popularity?
- Which is the party whose candidates are more likely to tweet? Why do you think that is the case?
- What are the topics candidates got more replies? Are there party differences (e.g. Labour candidates got more replies to some topics)? Why?

Prepare a proper write-up with more than 500 words.
